<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 790ms/step - loss: 0.6615 - accuracy: 0.6180 - val_loss: 0.6262 - val_accuracy: 0.7659
Epoch 2/10
26/26 [==============================] - 20s 761ms/step - loss: 0.5992 - accuracy: 0.7311 - val_loss: 0.6093 - val_accuracy: 0.6439
Epoch 3/10
26/26 [==============================] - 20s 762ms/step - loss: 0.5381 - accuracy: 0.7482 - val_loss: 0.6060 - val_accuracy: 0.6195
Epoch 4/10
26/26 [==============================] - 20s 757ms/step - loss: 0.4754 - accuracy: 0.7932 - val_loss: 0.4713 - val_accuracy: 0.8000
Epoch 5/10
26/26 [==============================] - 20s 754ms/step - loss: 0.3945 - accuracy: 0.8285 - val_loss: 0.5018 - val_accuracy: 0.7512
Epoch 6/10
26/26 [==============================] - 20s 758ms/step - loss: 0.3444 - accuracy: 0.8613 - val_loss: 0.2776 - val_accuracy: 0.9024
Epoch 7/10
26/26 [==============================] - 19s 742ms/step - loss: 0.3266 - accuracy: 0.8674 - val_loss: 0.2282 - val_accuracy: 0.9463

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.65.h5
26/26 - 20s - loss: 0.6713 - accuracy: 0.5852 - val_loss: 0.6516 - val_accuracy: 0.7805
Epoch 2/5

Epoch 00002: saving model to weights.02-0.63.h5
26/26 - 20s - loss: 0.6141 - accuracy: 0.7336 - val_loss: 0.6285 - val_accuracy: 0.6000
Epoch 3/5

Epoch 00003: saving model to weights.03-0.53.h5
26/26 - 20s - loss: 0.5812 - accuracy: 0.7543 - val_loss: 0.5325 - val_accuracy: 0.8195
Epoch 4/5

Epoch 00004: saving model to weights.04-0.45.h5
26/26 - 20s - loss: 0.4994 - accuracy: 0.7725 - val_loss: 0.4529 - val_accuracy: 0.8098
Epoch 5/5

Epoch 00005: saving model to weights.05-0.40.h5
26/26 - 20s - loss: 0.4569 - accuracy: 0.8078 - val_loss: 0.3990 - val_accuracy: 0.8634


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 20s - loss: 0.6811 - accuracy: 0.6058 - val_loss: 0.6718 - val_accuracy: 0.5659


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 20s - loss: 0.6643 - accuracy: 0.6034 - val_loss: 0.6440 - val_accuracy: 0.7707
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 22s - loss: 0.6109 - accuracy: 0.7664 - val_loss: 0.6852 - val_accuracy: 0.4488


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 21s - loss: 0.6741 - accuracy: 0.5876 - val_loss: 0.6464 - val_accuracy: 0.7171
Epoch 2/50
26/26 - 19s - loss: 0.6147 - accuracy: 0.6983 - val_loss: 0.5886 - val_accuracy: 0.6829
Epoch 3/50
26/26 - 19s - loss: 0.5815 - accuracy: 0.6946 - val_loss: 0.6865 - val_accuracy: 0.5268
Epoch 4/50
26/26 - 20s - loss: 0.5272 - accuracy: 0.7652 - val_loss: 0.5216 - val_accuracy: 0.7756
Epoch 5/50
26/26 - 19s - loss: 0.4603 - accuracy: 0.7944 - val_loss: 0.4761 - val_accuracy: 0.8146
Epoch 6/50
26/26 - 20s - loss: 0.4023 - accuracy: 0.8406 - val_loss: 0.3781 - val_accuracy: 0.8927
Epoch 7/50
26/26 - 19s - loss: 0.3589 - accuracy: 0.8637 - val_loss: 0.3059 - val_accuracy: 0.8976
Epoch 8/50
26/26 - 22s - loss: 0.3052 - accuracy: 0.8942 - val_loss: 0.2498 - val_accuracy: 0.9317
Epoch 9/50
26/26 - 19s - loss: 0.2775 - accuracy: 0.9075 - val_loss: 0.2111 - val_accuracy: 0.9317
Epoch 10/50
26/26 - 19s - loss: 0.2016 - accuracy: 0.9404 - val_loss: 0.2144 - val_accuracy: 0.9220
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 20s 772ms/step - loss: 0.6596 - accuracy: 0.5998 - val_loss: 0.7534 - val_accuracy: 0.4341
Epoch 2/5
26/26 [==============================] - 19s 743ms/step - loss: 0.6117 - accuracy: 0.7007 - val_loss: 0.5662 - val_accuracy: 0.7512
Epoch 3/5
26/26 [==============================] - 19s 750ms/step - loss: 0.5613 - accuracy: 0.7275 - val_loss: 0.5505 - val_accuracy: 0.7512
Epoch 4/5
26/26 [==============================] - 19s 750ms/step - loss: 0.4967 - accuracy: 0.7822 - val_loss: 0.4820 - val_accuracy: 0.8146
Epoch 5/5
26/26 [==============================] - 20s 750ms/step - loss: 0.4372 - accuracy: 0.8090 - val_loss: 0.3773 - val_accuracy: 0.8293


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.599757,0.661113,0.434146,0.753373
1,1,0.700730,0.612141,0.751219,0.566210
2,2,0.727494,0.562303,0.751219,0.550518
3,3,0.782238,0.496309,0.814634,0.481989
4,4,0.809002,0.437352,0.829268,0.377332


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 32s 1s/step - loss: 0.6811 - accuracy: 0.5401 - val_loss: 0.6742 - val_accuracy: 0.6293

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 19s 742ms/step - loss: 0.6635 - accuracy: 0.5973 - val_loss: 0.6680 - val_accuracy: 0.5951

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 22s 830ms/step - loss: 0.6539 - accuracy: 0.6229 - val_loss: 0.6659 - val_accuracy: 0.5610

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 751ms/step - loss: 0.6494 - accuracy: 0.6618 - val_loss: 0.6659 - val_accuracy: 0.5366

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 19s 746ms/step - loss: 0.6468

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 23s 901ms/step - loss: 0.6737 - accuracy: 0.5584 - val_loss: 0.6542 - val_accuracy: 0.5805
Epoch 2/50
26/26 [==============================] - 20s 761ms/step - loss: 0.6331 - accuracy: 0.6837 - val_loss: 0.6087 - val_accuracy: 0.7951
Epoch 3/50
26/26 [==============================] - 20s 754ms/step - loss: 0.5856 - accuracy: 0.7518 - val_loss: 0.5534 - val_accuracy: 0.7366
Epoch 4/50
26/26 [==============================] - 19s 750ms/step - loss: 0.5067 - accuracy: 0.8017 - val_loss: 0.4653 - val_accuracy: 0.8244
Epoch 5/50
25/26 [===========================>..] - ETA: 0s - loss: 0.4567 - accuracy: 0.7987
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 20s 774ms/step - loss: 0.4540 - accuracy: 0.8005 - val_loss: 0.4918 - val_accuracy: 0.7561
Epoch 6/50
26/26 [==============================] - 20s 753ms/step - loss: 0.3808 - accuracy: 0.8601 - val_loss: 0.3724 - 

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 4444), started 0:16:44 ago. (Use '!kill 4444' to kill it.)